<a href="https://colab.research.google.com/github/cmlg96/spatial-data-management-with-GEE/blob/main/Lab03_cm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Part 1: The code creates a centered interactive map, adds custom text in the lower right corner, loads a digital elevation model (DEM) from Earth Engine and displays it as a digital surface model (DSM) with a color palette.

In [ ]:
!pip install geemap

In [ ]:
!pip install earthengine-api

In [7]:
import geemap
import ee

In [9]:
ee.Authenticate()
ee.Initialize(project='my-project')

In [10]:
geemap.ee_initialize()

In [11]:
# Crear un mapa interactivo
m = geemap.Map(center=[42.3, -8.2], zoom=10, height="600px")

# Definir las coordenadas del polígono de la región de interés (ROI)
roi = ee.Geometry.Polygon([[[-8.64, 42.24], [-8.05, 42.24], [-8.05, 42.46], [-8.64, 42.46], [-8.64, 42.24]]])

# Agregar la región enmarcada en el mapa
m.addLayer(roi, {'color': 'red'}, 'Región enmarcada')

# Agregar texto al mapa
text = "Clara MR"
m.add_text(text, fontsize=20, position='bottomright')

# Cargar el DEM (Modelo Digital de Elevación) desde Earth Engine
DEM = ee.Image('CGIAR/SRTM90_V4')  # Modelo de elevación

# Seleccionar la capa de elevación y recortarla con el ROI
DSM = DEM.select('elevation').clip(roi)

# Parámetro de visualización del DSM
visParamsDSM = {
    'min': 0,
    'max': 4000,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

# Agregar la capa del DSM recortada al mapa
m.addLayer(DSM, visParamsDSM, 'DSM recortado')

# Mostrar el mapa interactivo
m




Map(center=[42.3, -8.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

#### Part 2: The code creates an interactive map centered on coordinates, defines a study geometry, calculates the Normalized Burn Vegetation Index (NBR) from pre-fire satellite imagery from the Sentinel-2 collection and visualizes this index with a color palette.


In [12]:
# Crear un mapa interactivo
m = geemap.Map(center=[42.3, -8.2], zoom=10, height="600px")

# Agregar texto al mapa
text = "Clara MR"
m.add_text(text, fontsize=20, position='bottomright')

# Geometría de estudio
geometry = ee.Geometry.Polygon([
    [[-8.64, 42.24], [-8.05, 42.24], [-8.05, 42.46], [-8.64, 42.46], [-8.64, 42.24]]
])

# Agregar el área de estudio enmarcada en rojo
m.addLayer(geometry, {'color': 'red'}, 'Área de estudio')

# Función para calcular NBR
def calculate_nbr(image):
    return image.normalizedDifference(['B8A', 'B12']).rename('NBR')

# Obtener la colección de imágenes post-incendio
img_post_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2017-10-21', '2017-11-01') \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']) \
    .filterBounds(geometry) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)

# Obtener fechas
dates_post = img_post_collection.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print('Fechas de las imágenes Post-Incendio:', dates_post.getInfo())

# Generar mosaico y cortar al área de estudio
img_post = img_post_collection.mosaic().multiply(0.0001).clip(geometry)

# Calcular NBR Post-Incendio y recortar al área de estudio
nbr_post = calculate_nbr(img_post).clip(geometry)

# Convertir el NBR a tipo entero (int16)
nbr_post_int = nbr_post.multiply(1000).int16()

# Configuración de visualización
vis_params = {
    'min': -1000,
    'max': 1000,
    'palette': ['red', 'white']
}

# Agregar capa NBR Post-Incendio al mapa
m.addLayer(nbr_post_int, vis_params, 'NBR Post-Incendio')

# Añadir controles al mapa
m.addLayerControl()

# Mostrar el mapa
m


Fechas de las imágenes Post-Incendio: ['2017-10-22']


Map(center=[42.3, -8.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…